# On linear programming with OR-tools and using OR-tools to solve an optimization problem with stochastic dominance constraints

### Nikolai Chow

## Overview

1. Linear programming with Google's OR-tools vs SciPy


2. A portfolio optimization problem with stochastic dominance constraints that can be solved by linear programing


3. An example in "Dentcheva, D., & Ruszczyński, A. (2003). Optimization with stochastic dominance constraints. SIAM Journal on Optimization." with Google's OR-tools and SciPy

## 1. Linear programming with Google's OR-tools vs SciPy

Google OR-Tools is a free and open-source software suite developed by Google. 

OR-Tools includes solvers for:

1. Linear and Mixed-Integer Programming


2. Constraint Programming 
    - How can N queens be placed on an NxN chessboard so that no two of them attack each other?
    - Create a schedule for four nurses over a three-day period


3. Graph Algorithms


### OR-tools vs SciPy (My Personal Considerations)


1. Speed of solving the problem

2. Time required for problem formulation

3. Code readability



#### Scipy

- [highs](https://highs.dev/#contact) (default)
- highs-ds 
- highs-ipm 
- interior-point (legacy)
- revised simplex (legacy) 
- simplex (legacy)  

The legacy methods are deprecated and will be removed in SciPy 1.11.0.

#### OR-tools

- GLOP
- [SCIP](https://scipopt.org/)

![MIPLIB2017.jpg](https://lh5.googleusercontent.com/9P8C7E0nMD7kaCN5H9-Ed__57ZMIozY8BlOclBA_AHdWNHXctLLGlRQqXkX2s-4xSds=w2400)

Source: http://plato.asu.edu/ftp/milp.html


### Example 1: Production Problem

The problem is:

$$
\begin{aligned}
\max_{x_1,x_2} \ & 3 x_1 + 4 x_2 \\
\mbox{subject to } \ & 2 x_1 + 5 x_2 \le 30 \\
& 4 x_1 + 2 x_2 \le 20 \\
& x_1, x_2 \ge 0 \\
\end{aligned}
$$

Source: https://python.quantecon.org/lp_intro.html

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from matplotlib.patches import Polygon
from scipy.optimize import linprog
from ortools.linear_solver import pywraplp
%matplotlib inline

### Using SciPy

In [2]:
# Construct parameters
c_ex1 = np.array([3, 4])

# Inequality constraints
A_ex1 = np.array([[2, 5],
                  [4, 2]])
b_ex1 = np.array([30,20])

# Solve the problem
# we put a negative sign on the objective as linprog does minimization
res_ex1 = linprog(-c_ex1, A_ub=A_ex1, b_ub=b_ex1, method='highs')

res_ex1

           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -27.5
       ineqlin:  marginals: array([-0.625 , -0.4375])
  residual: array([0., 0.])
         lower:  marginals: <MemoryView of 'ndarray' at 0x25fd918b380>
  residual: array([2.5, 5. ])
       message: 'Optimization terminated successfully.'
           nit: 2
         slack: array([0., 0.])
        status: 0
       success: True
         upper:  marginals: <MemoryView of 'ndarray' at 0x25fd918b1e0>
  residual: array([inf, inf])
             x: array([2.5, 5. ])

### Using OR-tools

In [3]:
# Declare the LP solver
solver = pywraplp.Solver.CreateSolver('GLOP')

# Create the variables
# NumVar for continuous variables; IntVar for integer variables; BoolVar for boolean variables.
x1 = solver.NumVar(0, solver.infinity(), 'x1')
x2 = solver.NumVar(0, solver.infinity(), 'x2')

# Objective function: 3x + 4y.
solver.Maximize(3*x1 + 4*x2)

# Inequality constraints
solver.Add(2*x1 + 5*x2 <= 30)
solver.Add(4*x1 + 2*x2 <= 20)

status = solver.Solve()

print('Solution:')
print('Objective value =', solver.Objective().Value())
print('x1 =', x1.solution_value())
print('x2 =', x2.solution_value())

Solution:
Objective value = 27.5
x1 = 2.4999999999999996
x2 = 5.0


### Example 2: Investment Problem

The problem is:

$$
\begin{aligned}
\max_{x} \ & 1.30 \cdot 3x_1 + 1.06 x_4 + 1.30 x_5 \\
\mbox{subject to } \ & x_1 + x_2 = 100,000\\
 & x_1 - 1.06 x_2 + x_3 + x_5 = 0\\
 & x_1 - 1.06 x_3 + x_4 = 0\\
 & x_2 \ge -20,000\\
 & x_3 \ge -20,000\\
 & x_4 \ge -20,000\\
 & x_5 \le 50,000\\
 & x_j \ge 0, \quad j = 1,5\\
 & x_j \ \text{unrestricted}, \quad j = 2,3,4\\
\end{aligned}
$$

Source: https://python.quantecon.org/lp_intro.html

### Using SciPy

In [4]:
# Construct parameters
rate = 1.06

# Objective function parameters
c_ex2 = np.array([1.30*3, 0, 0, 1.06, 1.30])

# Inequality constraints ??
A_ex2 = np.array([[1,  1,  0,  0,  0],
                  [1, -rate, 1, 0, 1],
                  [1, 0, -rate, 1, 0]])
b_ex2 = np.array([100000, 0, 0])

# Bounds on decision variables
bounds_ex2 = [(  0,    None),
              (-20000, None),
              (-20000, None),
              (-20000, None),
              (  0,   50000)]

# Solve the problem
res_ex2 = linprog(-c_ex2, A_eq=A_ex2, b_eq=b_ex2,
                  bounds=bounds_ex2, method='revised simplex')

res_ex2

     con: array([ 1.45519152e-11, -2.18278728e-11,  0.00000000e+00])
     fun: -141018.24349792692
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 24927.75474306,  75072.24525694,   4648.8252293 , -20000.        ,
        50000.        ])

### Using OR-tools

In [5]:
# Declare the LP solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Create the variables
x_s = [solver.NumVar(0, solver.infinity(), 'x1')]
x_s = x_s + [solver.NumVar(-20000, solver.infinity(),f'x{i}') for i in range(2,5)]
x_s = x_s + [solver.NumVar(0, 50000.00, 'x5')]

# Objective function
solver.Maximize(1.30*3*x_s[0] + 1.06*x_s[3] + 1.30*x_s[4])

# Constraints
solver.Add(x_s[0] + x_s[1] == 100000)
solver.Add(x_s[0] - 1.06*x_s[1] + x_s[2] + x_s[4] == 0)
solver.Add(x_s[0] - 1.06*x_s[2] + x_s[3] == 0)

status = solver.Solve()

print('Solution:')
print('Objective value =', solver.Objective().Value())
for i in range(len(x_s)):
    print(f'x{i+1} = {x_s[i].solution_value()}')

Solution:
Objective value = 141018.24349792695
x1 = 24927.75474305819
x2 = 75072.24525694181
x3 = 4648.825229300174
x4 = -20000.000000000007
x5 = 49999.999999999956


## 2. A portfolio optimization problem with stochastic dominance constraints



### 2.1. Review of stochastic dominance

Let random variable $X\in \mathcal{L}^1(\Omega,\mathcal{F},P)$ such that $X: \Omega \rightarrow \mathbb{R}$, where $(\Omega,\mathcal{F},P)$ is the probability space.

The cumulative distribution function (CDF) of $X$ is defined as:

$$F(\eta) = P[X \leq \eta]$$

The second CDF of $X$ is defined as:

$$F_2(\eta) = \int^\eta_{-\infty}F(t)dt$$ for $\eta \in R$.


Similarly, for random variable $Y\in \mathcal{L}^1(\Omega,\mathcal{F},P)$ such that $Y: \Omega \rightarrow \mathbb{R}$, we have $G(\eta)$ and $G_2(\eta)$.

With the above setting, stochastic dominance relationships are defined as:

1. $G$ is said to be first order stochastic dominated by $F$  ($F\succeq_1 G$) if

$$F(\eta)\leq G(\eta) \;\text{for all}\; \eta \in  \mathbb{R}.$$


2. $G$ is said to be second order stochastic dominated by $F$  ($F\succeq_2 G$) if

$$F_2(\eta)\leq G_2(\eta) \;\text{for all}\; \eta \in  \mathbb{R}.$$

###  Why should we care?

There are several reasons why stochastic dominance is a valuable tool in economics and finance. Here are two key theorems from Hadar and Russell (1969), Hanoch and Levy (1975), and Tesfatsion (1976) that highlight the importance of stochastic dominance:

1. The relation $F\succeq_1 G$ is equivalent to $E[u(X)]\geq E[u(Y)]$
for all utility function $u$ with $u'\geq 0$


2. The relation $F\succeq_2 G$ is equivalent to $E[u(X)]\geq E[u(Y)]$
for all utility function $u$ with $u'\geq 0$ and $u''\leq 0$

### One direction of stochastic dominance research is to find out the way to form SSD-efficient portfolios. In general, this is not an easy task. 

### 2.2. A portfolio optimization problem



Let $R_1,..., R_N $ be random returns for assets $1,...,N$,  with joint cumulative distribution function $\mathcal{F}(\textbf{r})$ where $\textbf{r}\in R_1 \times ... \times R_N$. Assume that $E[|R_n|]< \infty$ for all $n=1,...,N$, and further let $w_1,...,w_N$ as portfolio weights on assets $1,...,N$. Without short selling, the set of possible asset allocations is defined as:

$$W=\Big\{w\in \mathbb{R}^N: \sum^N_{i=1} w_i=1, w_n\geq 0 \;\; \text{for}\;\; n=1,...,N\Big\}.$$

The CDF $\mathcal{F}_w(\eta)$ for asset allocation $w\in W$ is defined as:

$$\mathcal{F}_w(\eta)=\int_{\{\textbf{r}^Tw\leq \eta\}}d\mathcal{F}(\textbf{r})$$

where the corresponding portfolio return is denoted as $R_w=R_1w_1+...+R_Nw_N$. For empirical purposes, we assume that by selecting appropriate values for $a$ and $b$, we can ensure that the $\mathcal{F}_w(a) = 0$ and $\mathcal{F}_w(b) = 1$ for all $w \in W$.

Letting $Y$ as target random return with finite expected value and corresponding CDF $G$, such that $G(a)=0$ and $G(b)=1$. Under these settings, one could consider the following problem: 

\begin{align}
    \max E[R_w] \nonumber\\
    \text{subject to} \;\; \mathcal{F}_w\succeq_{2} G\nonumber\\
    w \in W 
\end{align}
where $E[R_w]$ is the expected portoflio return.

Solving the problem can be challenging since it involves computing intergals of CDF $\mathcal{F}_w(\eta)$ and $\mathcal{F}_w(\eta)$ will vary significantly when the weights on assets are different.

To address this, Dentcheva and Ruszczyński (2003) use a three steps approach:

1. Transform the problem into an equivalent problem that doesn't require computing intergals of marginal CDF.
2. Show that if $Y$ has a discrete distribution with known realizations, we don't need to compare intergals of CDF for all $\eta \in [a,b]$.
3. Further transform the problem into a linear programming problem.

### 2.3. An equivalent problem

Bawa et al.(1985) states the following:

$$F_2(\eta)\leq G_2(\eta) \;\text{for all}\; \eta \in [a,b]$$

$$\Longleftrightarrow$$

\begin{aligned}
E[(\eta - X)_+]\leq E[(\eta -Y)_+] \ & \ \ \text{for all} \ \eta\in [a,b]
\end{aligned}

where $(\eta - X)_+=\max\{\eta - X,0\}$. $E[(\eta - X)_+]$ is the lower partial moment of X and $E[(\eta -Y)_+]$ is the lower partial moment of Y. 

Using this idea, one could transform the original problem into an equivalent problem as: 

\begin{align}
    \max &\; \;\;  E[R_w] \nonumber\\
    \text{subject to} 
    &\;\;\; E[(\eta - R_w)_+]\leq E[(\eta -Y)_+] \ & \ \ \text{for all} \ \eta\in [a,b]\\ 
   &\;\;\;  w \in W. \nonumber
\end{align}

### 2.4. Show that we don't need to compare intergals of CDFs for all $\eta \in [a,b]$

Further assume Y has a discrete distribution with realizations $y_i$, $i =1,...,m$ then Dentcheva and Ruszczyński (2003) shows:

\begin{aligned}
E[(\eta - R_w)_+]\leq E[(\eta -Y)_+] \ & \ \ \text{for all} \ \eta\in [a,b]
\end{aligned}

$$\Longleftrightarrow$$

\begin{aligned}
E[(y_i - R_w)_+]\leq E[(y_i -Y)_+], \ & \ \ \       i=1,...,m 
\end{aligned}

This equivalent relationship allow us to compare these partial moments only for a finite number of return levels.

### 2.5. Further transform the problem into a linear programming problem

Assume there are finitly many elementary events such that $\Omega=\{\nu_1,...,\nu_m\}$ and introduce a decision vector $S:[a,b] \times \Omega \rightarrow  \mathbb{R}$ , then $E[(\eta - R_w)_+]$ will be the solution of the following linear programming problem:


\begin{align}
\min_{S(\eta,\nu)} &\;\;\;E[ S(\eta,\nu)] \nonumber \\
\text{subject to} &\;\;\; S(\eta,\nu_i) \geq \eta - R_w(\nu_i) &\text{for} \;i=1,...,m \nonumber \\
&\;\;\; S(\eta,\nu_i) \geq 0 &\text{for} \;i=1,...,m
\end{align}


Since $E[ S(\eta,\nu)] = P[\{\nu_1\}] S(\eta,\nu_1) + ... + P[\{\nu_m\}] S(\eta,\nu_m)$, minimizing $E[ S(\eta,\nu)]$ is equivalent to minimizing $S(\eta,\nu_i)$ for all $\nu_i \in \Omega$ and  minimizing $S(\eta,\nu_i)$ will enforce the solution $S(\eta,\nu_i)^*$ equal to $max\{ \eta - R_w(\nu_i),0\}$. Therefore $E[ S(\eta,\nu)^*]=E[(\eta - R_w)_+]$. 

Using this idea, the original problem could be further transformed into the following:

\begin{align}
\max  &\;\;\; E[R_w]\\
\text{subject to}  &\;\;\; S(y_i,\nu_k)\geq y_i -R_w(\nu_i) &i=1,...,m, k=1,..., m\nonumber\\
&\;\;\; E[S(y_i,\nu)]\leq E[(y_i-Y)_+] &i=1,...,m\nonumber\\
&\;\;\;S(y_i,\nu_k)\geq 0 &i=1,...,m, k=1,..., m\nonumber\\
&\;\;\;w \in W \nonumber\\
\end{align} 

Kindly note that in this problem, the solution $S(η,\nu_i)^{**}$ will not necessarily enforce $E[S(y_i,\nu)^{**}]$ to be equal to $E[(y_i-R_w)_+]$, since we are not explicitly minimizing $E[S(\eta,\nu)]$. However, we know that $E[(y_i-R_w)_+] \leq E[S(y_i,\nu)^{**}]$, therefore solving the above problem will ensure that $E[(y_i-R_w)_+] \leq E[(y_i-Y)_+]$.


Before we show the final product, we need some more notations.  

- Let $p_k=P[\{\nu_k\}]$, $r_{nk}=R_n(\nu_k), s_{ik}=S(y_i,\nu_k)$, and $v_i=E[(y_i-Y)_+]$.

Then the original problem could be formulated as the following linear programming problem:

\begin{aligned}
\max_{w,s} &\; \sum_{k=1}^{m}\sum_{n=1}^{N} p_kr_{nk}w_n\nonumber\\
\text{subject to}&\; \sum_{n=1}^{N} r_{nk}w_n+s_{ik}\geq y_i, &i=1,...,m, k=1,..., m\nonumber\\
&\sum_{k=1}^{m} p_k s_{ik}\leq v_i, &i=1,...,m\nonumber\\
&s_{ik}\geq0&i=1,...,m, k=1,..., m\nonumber\\
&\sum^N_{n=1} w_n =1\nonumber\\
&w_n \geq 0 &n=1,...,N \nonumber\\
\end{aligned}

## 3. An example in Dentcheva and Ruszczyński (2003)  with Python

Now we turn to the an example at p.562 in Dentcheva and Ruszczyński (2003). 

In [6]:
#Number of w is N=8 and s = m*m = 22*22 = 484 is 492
# intialise data of lists.

data = {'Asset 1':[  7.5,   8.4,  6.1,  5.2,  5.5,
                     7.7,  10.9, 12.7, 15.6, 11.7,
                     9.2,  10.3,    8,  6.3,  6.1,
                     7.1,   8.7,    8,  5.7,  3.6,  3.1,  4.5],
        'Asset 2':[ -5.8,     2,  5.6, 17.5,  0.2, 
                    -1.8,  -2.2, -5.3,  0.3, 46.5,
                    -1.5,  15.9, 36.6, 30.9, -7.5,
                     8.6,  21.2,  5.4, 19.3,  7.9, 21.7,-11.1],
        'Asset 3':[-14.8, -26.5, 37.1, 23.6, -7.4,
                     6.4,  18.4, 32.3, -5.1, 21.5,
                    22.4,   6.1, 31.6, 18.6,  5.2,
                    16.5,  31.6, -3.2, 30.4,  7.6,   10,  1.2],
        'Asset 4':[-18.5, -28.4, 38.5, 26.6, -2.6,
                     9.3,  25.6, 33.7, -3.7, 18.7,
                    23.5,     3, 32.6, 16.1,  2.3,
                    17.9,  29.2, -6.2, 34.2,    9, 11.3, -0.1],
        'Asset 5':[-30.2, -33.8, 31.8,   28,  9.3, 
                    14.6,  30.7, 36.7,   -1, 21.3,
                    21.7,  -9.7, 33.3,  8.6, -4.1,
                    16.5,  20.4,  -17, 59.4, 17.4, 16.2, -3.2],
        'Asset 6':[  2.3,   0.2, 12.3, 15.6,    3,  
                     1.2,   2.3,  3.1,  7.3, 31.1,
                       8,    15, 21.3, 15.6,  2.3,
                     7.6,  14.2,  8.3, 16.1,  7.6,   11, -3.5],
        'Asset 7':[-14.9, -23.2, 35.4,  2.5, 18.1, 
                    32.6,   4.8, 22.6, -2.3, -1.9,
                    23.7,   7.4, 56.2, 69.4, 24.6,
                    28.3,  10.5,-23.4, 12.1,-12.2, 32.6,  7.8],
        'Asset 8':[ 67.7,  72.2,  -24,   -4,   20, 
                    29.5,  21.2, 29.6,-31.2,  8.4,
                   -12.8, -17.5,  0.6, 21.6, 24.4,
                   -13.9,  -2.3, -7.8, -4.2, -7.4, 14.6,   -1]}

Number of variables = 492

Number of constraints = 507

In [7]:
# Create DataFrame
df = pd.DataFrame(data)

# Number of assets
NumA = len(df.columns)
# Number of states
NumS = len(df)

# List for col index
cols = [f'Asset {i}' for i in range(1, NumA+1)]

We start by setting up the target portfolio. It is just a equally weighted portfolio.

In [8]:
# Create an array of weights, each set to 1/NumA
weight = np.full(NumA, 1/NumA)

# Calculate the weighted average of the columns in the dataframe
y_k = np.average(np.asarray(df[cols]), weights=weight, axis=1)

# Calculate the probability of each state 
p_k = 1/NumS

#v_i are patial moments
v_i = np.array([])


for x in range(NumS):
    yp = y_k[x]-y_k
    yp[yp < 0] = 0
    v_i = np.append(v_i, np.mean(yp))

### 3.1. Using Scipy

The goal is use the following line at the end:

res_ex1 = linprog(-c_ex1, A_ub=A_ex1, b_ub=b_ex1, A_eq=A_ex2, b_eq=b_ex2,
                  bounds=bounds_ex1, method='highs')



### 3.1.1. Pain in the ass (or Setting up objective and constraints)

Here we setup the parameters for the objective function:

In [9]:
# start = time.time()

# Calculate the probability of each state times the total return of assets
#e.g pk(r11+r21+...), pk(r21+r22+...)
pk_S_rnk = p_k * df[cols].sum()

# Convert the resulting series to a list
pk_S_rnk = pk_S_rnk.values.tolist()

#The s_ik variables
s_ik = [0]*NumS*NumS

# Setting the objective for linear programing 
# Adding the list together
c_ex1 = np.array(pk_S_rnk + s_ik)  

Now we turn to inequality constraints:

In [10]:
r_list = df.values.tolist()

#Copy the return for m states
r_list = r_list * NumS 

# Stack two below matrices together
b1 = np.array(r_list)
b2 = np.identity(NumS*NumS)

# The first line of the constraints
A_ex1_1 = np.column_stack((b1, b2))

In [11]:
#b3 is for w_i
b3 = np.array(df.values.tolist())*0
b4 = np.zeros((NumS,NumS*NumS))

for x in range(NumS):
   b4[x,NumS*x:NumS*(x+1)] = 1  
    
A_ex1_2 = p_k*np.column_stack((b3, b4))

# The inequality constraints for linear programing
A_ex1 = np.row_stack((-A_ex1_1,A_ex1_2))

In [12]:
#Working on the bounds of inequality constraints

# Bounds of inequality constraints used for lP
b_ex1 = np.array([])
for i in range(NumS):
    for x in range(NumS):
        b_ex1 = np.append(b_ex1, y_k[i])


b_ex1 = np.append(-b_ex1,v_i)

Last but not least, the equality constraint:

In [13]:
# Working on equality constraint
A_ex2 = np.array([np.append(np.ones(NumA), np.zeros(NumS*NumS))])

#print(A_ex2)
b_ex2 = np.array([1])

# Bounds on decision variables
bounds_ex1 =[]
for x in range(NumA+NumS*NumS):
    bounds_ex1 = bounds_ex1 + [(0, None)]

### 3.1.2 Solving the linear programming problem
Now, we can use the solver to figure out the best solution of our linear programming problem:

In [14]:
# Solve the problem

start = time.time()
# res_ex1 = linprog(-c_ex1, A_ub=A_ex1, b_ub=b_ex1, A_eq=A_ex2, b_eq=b_ex2,
#                   bounds=bounds_ex1, method='revised simplex')
res_ex1 = linprog(-c_ex1, A_ub=A_ex1, b_ub=b_ex1, A_eq=A_ex2, b_eq=b_ex2,
                  bounds=bounds_ex1, method='highs')
end = time.time()

print("The optimal portfolio has the form")
print(res_ex1.x[0:8])

print("Execution time: ", end - start)

The optimal portfolio has the form
[0.         0.         0.06803595 0.18800344 0.         0.39137554
 0.23092417 0.12166089]
Execution time:  0.023485183715820312


### 3.2 Using OR-tools

In [15]:
# start = time.time()
# Instantiate a SCIP solver and naming it.
solver = pywraplp.Solver.CreateSolver('SCIP')


# Declare arrays to hold our variables.
# The below code will store information to objective "solver". 
# Make sure that you didn't store the same information repeatly in the "solver"
weight_A = [solver.NumVar(0.0, solver.infinity(), f'w_{i+1}') for i in range(NumA)]
s_ik = [solver.NumVar(0.0, solver.infinity(), f's_{i+1}_{k+1}') for i in range(NumS) for k in range(NumS)]


#First inequality constraint
for i in range(NumS):
    for k in range(NumS):
        solver.Add(sum(df[cols[n]][k]*weight_A[n] for n in range(NumA)) + s_ik[i*NumS+k] >= y_k[i])

#Second inequality constraint
for i in range(NumS):
    solver.Add(sum(p_k*s_ik[i*NumS+k] for k in range(NumS)  ) <= v_i[i])
    
# Equality constraint
solver.Add(sum(weight_A[n] for n in range(NumA) ) == 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000025FDAC3CD20> >

In [16]:
solver.Maximize(sum(sum(p_k*df[cols[n]][k]*weight_A[n] for n in range(NumA)) for k in range(NumS)))

In [17]:
# Solve the system.
start = time.time()
status = solver.Solve()
end = time.time()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for n in range(NumA):
        print(f'w{n+1} =', weight_A[n].solution_value())
else:
    print('The problem does not have an optimal solution.')


print("Execution time: ", end - start)

Solution:
Objective value = 11.008198995664435
w1 = 0.0
w2 = 0.0
w3 = 0.06803595208792393
w4 = 0.18800344323438142
w5 = 0.0
w6 = 0.3913755414860866
w7 = 0.23092417350675995
w8 = 0.12166088968484806
Execution time:  0.08620619773864746


## Reference

Atkinson, A. B. (1970). On the measurement of inequality. Journal of economic theory, 2(3), 244-263.

Bawa, V. S., Bodurtha Jr, J. N., Rao, M. R., & Suri, H. L. (1985). On determination of stochastic dominance optimal sets. The Journal of Finance, 40(2), 417-431.

Cho, Y. H., Linton, O., & Whang, Y. J. (2007). Are there Monday effects in stock returns: A stochastic dominance approach. Journal of Empirical Finance, 14(5), 736-755.

Chui, D., Cheng, W. W., Chow, S. C., & Ya, L. I. (2020). Eastern Halloween effect: A stochastic dominance approach. Journal of International Financial Markets, Institutions and Money, 68, 101241.

Dentcheva, D., & Ruszczyński, A. (2003). Optimization with stochastic dominance constraints. SIAM Journal on Optimization, 14(2), 548-566.

Fang, Y., & Post, T. (2022). Optimal portfolio choice for higher-order risk averters. Journal of Banking & Finance, 137, 106429.

Guerre, E., Perrigne, I., & Vuong, Q. (2009). Nonparametric identification of risk aversion in first‐price auctions under exclusion restrictions. Econometrica, 77(4), 1193-1227.

Hadar, J., & Russell, W. R. (1969). Rules for ordering uncertain prospects. The American economic review, 59(1), 25-34.

Hanoch, G., & Levy, H. (1975). The efficiency analysis of choices involving risk. In Stochastic Optimization Models in Finance (pp. 89-100). Academic Press.

Levy, H. (2016). Stochastic dominance: Investment decision making under uncertainty. New York: Springer.

Maasoumi, E., Millimet, D. L., & Rangaprasad, V. (2005). Class size and educational policy: Who benefits from smaller classes?. Econometric Reviews, 24(4), 333-368.

Rothschild, M., & Stiglitz, J. E. (1978). Increasing risk: I. A definition. In Uncertainty in Economics (pp. 99-121). Academic Press.

Tesfatsion, L. (1976). Stochastic dominance and the maximization of expected utility. The Review of Economic Studies, 43(2), 301-315.